In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd

In [2]:
df=pd.read_csv('G:\TENSOR\spam_ham_dataset.csv')

In [3]:
df

,label,text,label_num
0,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,spam,"Subject: photoshop , windows , office . cheap ...",1
4,ham,Subject: re : indian springs\r\nthis deal is t...,0
...,...,...,...
5166,ham,Subject: put the 10 on the ft\r\nthe transport...,0
5167,ham,Subject: 3 / 4 / 2000 and following noms\r\nhp...,0
5168,ham,Subject: calpine daily gas nomination\r\n>\r\n...,0
5169,ham,Subject: industrial worksheets for august 2000...,0


In [4]:
df.groupby('label').describe()

label_num                                   
          count mean  std  min  25%  50%  75%  max
label                                             
ham      3672.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
spam     1499.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0

In [5]:
df['label'].value_counts()

ham     3672
spam    1499
Name: label, dtype: int64

In [6]:
df_spam=df[df['label']=='spam']
df_spam.shape

(1499, 3)

In [7]:
df_ham=df[df['label']=='ham']
df_ham.shape

(3672, 3)

In [8]:
df_ham_under=df_ham.sample(df_spam.shape[0])

In [9]:
df_ham_under.shape

(1499, 3)

In [10]:
df_balance=pd.concat([df_ham_under,df_spam])
df_balance.shape

(2998, 3)

In [11]:
df_balance['label'].value_counts()

ham     1499
spam    1499
Name: label, dtype: int64

In [12]:
df_balance['binary']=df_balance['label'].apply(lambda x: 1 if x=='spam' else 0)

In [13]:
df_balance.sample(5)

,label,text,label_num,binary
4897,spam,Subject: re : hi suzette\r\nthe secret on how ...,1,1
1473,spam,Subject: dating service for nauuughty minded p...,1,1
4900,ham,"Subject: hpl nom for may 1 , 2001\r\n( see att...",0,0
470,spam,Subject: a huge load inside her pussy .\r\nemb...,1,1
5039,spam,Subject: just kidding\r\nif image above doesnt...,1,1


In [14]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df_balance['text'],df_balance['label_num'],random_state=42,stratify=df_balance['label_num'])

In [15]:
X_train.head(3)

3145    Subject: can ' t get it up\r\nstop wasting mon...
3244    Subject: want to make more money ?\r\norder co...
798     Subject: txu fuels / sds nomination for novemb...
Name: text, dtype: object

In [16]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [17]:
def get_sentence(sentence):
    preprocess_text=bert_preprocess(sentence)
    return bert_encoder(preprocess_text)['pooled_output']
get_sentence([
    "500$ discount. hurry up", 
    "Bhavin, are you up for a volleybal game tomorrow?"

])

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.84351707, -0.5132728 , -0.8884573 , ..., -0.74748874,
        -0.7531474 ,  0.91964495],
       [-0.87208354, -0.50543964, -0.9444667 , ..., -0.858475  ,
        -0.7174534 ,  0.88082975]], dtype=float32)>

In [18]:
e=get_sentence([
    "banana", 
    "grapes",
    "mango",
    "jeff bezos",
    "elon musk",
    "bill gates"
    
])

In [19]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity([e[0]],[e[1]])

array([[0.99110866]], dtype=float32)

In [20]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [21]:
len(X_train)

2248

In [22]:
me=([
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
     tf.keras.metrics.Precision(name='precision'),
     tf.keras.metrics.Recall(name='recall')
    
])
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=me)


In [23]:
model.fit(X_train,y_train,epochs=3)

Epoch 1/3
71/71 [==============================] - 434s 6s/step - loss: 0.6425 - accuracy: 0.6125 - precision: 0.6246 - recall: 0.5641
Epoch 2/3
71/71 [==============================] - 394s 6s/step - loss: 0.5622 - accuracy: 0.7126 - precision: 0.7123 - recall: 0.7135
Epoch 3/3
71/71 [==============================] - 430s 6s/step - loss: 0.5151 - accuracy: 0.7536 - precision: 0.7659 - recall: 0.7304


In [24]:
model.evaluate(X_test,y_test)

24/24 [==============================] - 136s 6s/step - loss: 0.5080 - accuracy: 0.7493 - precision: 0.7332 - recall: 0.7840


[0.5079707503318787,
 0.7493333220481873,
 0.7331671118736267,
 0.7839999794960022]

In [29]:
y_pred=model.predict(X_test)
y_pred=y_pred.flatten()
# import numpy as np
# y_pred=np.where(y_pred>1,0.5,0)
# y_pred
# from sklearn.metrics import confusion_matrix,clasification_report
# cm=confusion_matrix(X_test,y_pred)
# cm
# from matplotlib import pyplot as plt
# import seaborn as sn
# sn.heatmap(cm, annot=True, fmt='d')
# plt.xlabel('Predicted')
# plt.ylabel('Truth')

In [30]:
import numpy as np
y_pred = np.where(y_pred > 0.5, 1, 0)
y_pred=model.predict(X_test)
y_pred=np.round(y_pred)

In [36]:
from sklearn.metrics import confusion_matrix,classification_report
cm=confusion_matrix(y_test,y_pred)
cm


array([[268, 107],
       [ 81, 294]], dtype=int64)

In [37]:
len(X_test)

750

In [38]:
from matplotlib import pyplot as plt
import seaborn as sn
sn.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')

ModuleNotFoundError: No module named 'seaborn'

In [39]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.77      0.71      0.74       375
           1       0.73      0.78      0.76       375

    accuracy                           0.75       750
   macro avg       0.75      0.75      0.75       750
weighted avg       0.75      0.75      0.75       750



In [40]:
reviews = [
    'Enter a chance to win $5000, hurry up, offer valid until march 31, 2021',
    'You are awarded a SiPix Digital Camera! call 09061221061 from landline. Delivery within 28days. T Cs Box177. M221BP. 2yr warranty. 150ppm. 16 . p pÂ£3.99',
    'it to 80488. Your 500 free text messages are valid until 31 December 2005.',
    'Hey Sam, Are you coming for a cricket game tomorrow',
    "Why don't you wait 'til at least wednesday to see if you get your ."
]
model.predict(reviews)

array([[0.50672615],
       [0.5561883 ],
       [0.4838251 ],
       [0.7823489 ],
       [0.87310815]], dtype=float32)